In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import pandas as pd


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.9/site-packages/transformers/__init__.py)

In [14]:
data = pd.read_csv("/workspaces/TestTecnique/finetuning/joueurs_dataset.csv", sep=",")
data.head(1)

,input,output
0,Ce joueur a 20 ans et évolue au poste de Arriè...,Le poste d'Arrière gauche semble bien adapté à...


In [18]:
#   model  de base a fin-tuner : TinyLLaMA 
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right",)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
model.to("cpu")

In [ ]:
txt = """<|system|>
          En tant qu'expert en analyse de statistques de football dis moi le poste du joueur est adapté et recommande des exercises physique pour ammeliorer ses perfomances.
        <|user|>
          Ce joueur a 20 ans et évolue au poste de Arrière gauche. Il a disputé 20 matchs, marqué 22 but(s) et délivré 18 passe(s) décisive(s).
        <|assistant|>

        """
tokens = tokenizer(txt, return_tensors="pt")['input_ids'].to("cpu")
op = model.generate(tokens, max_new_tokens=200)
print(tokenizer.decode(op[0]))

<s> ###SYSTEM: En tant qu'expert en analyse de statistques de football dis moi le poste du joueur est adapté et recommande des exercises physique pour ammeliorer ses perfomances

###INPUT: Ce joueur a 20 ans et évolue au poste de Arrière gauche. Il a disputé 20 matchs, marqué 22 but(s) et délivré 18 passe(s) décisive(s).

###OUTPUT: Ce joueur a une moyenne de 0,8 buts par match et une moyenne de 0,8 passe(s) décisive(s) par match. Il a une moyenne de 0,8 buts par match et une moyenne de 0,8 passe(s) décisive(s) par match.

###SYSTEM: En tant qu'expert en analyse de statistques de football dis moi le poste du joueur est adapté et recommande des exercises physique pour ammeliorer ses performances en tant que joueur de l'équipe.

###INPUT: Ce joueur a 20 ans et évolue au poste de Arrière gauche. Il a disputé 20 matchs, marqué 22 but(s) et délivré 18 passe(s) décisive(s).

###OUTPUT: Ce joueur a une moyenne de 0


In [ ]:


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023
None


In [29]:
#preparation de la data a entrainée
def format_dataset(data_point):
    prompt = f"""###SYSTEM: En tant qu'expert en analyse de statistques de football dis moi le poste du joueur est adapté et recommande des exercises physique pour ammeliorer ses perfomances.

###INPUT: {data_point['input']}

###OUTPUT: {data_point['output']}
"""
    tokens = tokenizer(prompt,
        truncation=True,
        max_length=256,
        padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens

In [ ]:
# cahrgement de ma data qui etaiten csv en format hunging face
dataset = load_dataset("csv", data_files="/workspaces/TestTecnique/finetuning/joueurs_dataset.csv",split="train")
print(dataset[0])
dataset = dataset.map(format_dataset)

print(dataset[0])


{'input': 'Ce joueur a 20 ans et évolue au poste de Arrière gauche. Il a disputé 20 matchs, marqué 22 but(s) et délivré 18 passe(s) décisive(s).', 'output': "Le poste d'Arrière gauche semble bien adapté à ce joueur jeune et dynamique avec une forte contribution offensive. Conseils : travaillez la vitesse de sprint, l’endurance aérobique et les centres en mouvement. Renforcez les quadriceps et les ischio-jambiers avec du squat et du gainage latéral."}


Map: 100%|██████████| 150/150 [00:00<00:00, 1505.85 examples/s]

{'input': 'Ce joueur a 20 ans et évolue au poste de Arrière gauche. Il a disputé 20 matchs, marqué 22 but(s) et délivré 18 passe(s) décisive(s).', 'output': "Le poste d'Arrière gauche semble bien adapté à ce joueur jeune et dynamique avec une forte contribution offensive. Conseils : travaillez la vitesse de sprint, l’endurance aérobique et les centres en mouvement. Renforcez les quadriceps et les ischio-jambiers avec du squat et du gainage latéral.", 'input_ids': [1, 835, 14816, 1254, 12665, 29901, 1174, 11172, 439, 29915, 735, 10700, 427, 16455, 344, 316, 12997, 1912, 316, 5733, 766, 19601, 454, 23277, 868, 23009, 707, 7744, 29948, 634, 27878, 13990, 553, 24472, 3476, 267, 4824, 1387, 1671, 626, 12873, 1611, 261, 3999, 23895, 2480, 778, 29889, 13, 13, 2277, 29937, 1177, 12336, 29901, 7747, 23009, 263, 29871, 29906, 29900, 6063, 634, 29378, 434, 782, 23277, 316, 826, 9318, 26003, 29889, 1720, 263, 8937, 29948, 29871, 29906, 29900, 1993, 29879, 29892, 1766, 17851, 29871, 29906, 29906, 5

In [ ]:
#  gardons uniquement la data que le model comprend (les sequences de token)
dataset = dataset.remove_columns(['input','output'])
print(dataset)

In [ ]:
print(tokenizer.decode(dataset[0]['input_ids']))

{'input_ids': [1, 835, 14816, 1254, 12665, 29901, 1174, 11172, 439, 29915, 735, 10700, 427, 16455, 344, 316, 12997, 1912, 316, 5733, 766, 19601, 454, 23277, 868, 23009, 707, 7744, 29948, 634, 27878, 13990, 553, 24472, 3476, 267, 4824, 1387, 1671, 626, 12873, 1611, 261, 3999, 23895, 2480, 778, 29889, 13, 13, 2277, 29937, 1177, 12336, 29901, 7747, 23009, 263, 29871, 29906, 29900, 6063, 634, 29378, 434, 782, 23277, 316, 826, 9318, 26003, 29889, 1720, 263, 8937, 29948, 29871, 29906, 29900, 1993, 29879, 29892, 1766, 17851, 29871, 29906, 29906, 541, 29898, 29879, 29897, 634, 1437, 17843, 5606, 29871, 29896, 29947, 21714, 29898, 29879, 29897, 1437, 3476, 573, 29898, 29879, 467, 13, 13, 2277, 29937, 12015, 12336, 29901, 951, 23277, 270, 29915, 1433, 9318, 26003, 3031, 569, 6079, 7744, 29948, 818, 2257, 23009, 17222, 634, 4292, 1387, 2535, 1597, 19624, 11896, 1283, 6270, 29889, 15946, 2719, 584, 6427, 8284, 29920, 425, 325, 3246, 344, 316, 269, 2158, 29892, 301, 30010, 355, 18541, 263, 10651, 2

In [47]:
train_dataset , test_dataset = dataset.train_test_split(test_size=0.1).values()
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 135
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15
})


In [56]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer = Trainer(
                    model = model, 
                    train_dataset = train_dataset, 
                    eval_dataset = test_dataset,
                    tokenizer = tokenizer, 
                    data_collator = data_collator, 

                    args = TrainingArguments(
                        output_dir="./training",
                        remove_unused_columns=False,
                        per_device_train_batch_size=2,
                        gradient_checkpointing=True,
                        gradient_accumulation_steps=4,
                        max_steps=100,
                        learning_rate=2.5e-5, 
                        logging_steps=5,
                        fp16=True,
                        optim="adamw_torch",
                        save_strategy="steps",     
                        save_steps=50,             
                        # evaluation_strategy="steps",
                        eval_steps=5,              
                        do_eval=True,
                        label_names= ['input_ids', 'attention_mask', 'labels'],
                        report_to = "none",
                        
                ))

/tmp/ipykernel_16137/3032920559.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [57]:
trainer.train()

/usr/local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
5,2.580700
10,2.505500
15,2.501200
20,2.419300
25,2.391000
30,2.331500
35,2.247500
40,2.248000
45,2.186000
50,2.138900


/usr/local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=100, training_loss=2.1645806980133058, metrics={'train_runtime': 8951.2147, 'train_samples_per_second': 0.089, 'train_steps_per_second': 0.011, 'total_flos': 1264639231918080.0, 'train_loss': 2.1645806980133058, 'epoch': 5.882352941176471})

In [1]:
txt = """
<|system|>
  En tant qu'expert en analyse de statistques de football dis moi le poste du joueur est adapté et recommande des exercises physique pour ammeliorer ses perfomances.</s>

<|user|>
  Ce joueur a 20 ans et évolue au poste de Arrière gauche. Il a disputé 20 matchs, marqué 22 but(s) et délivré 18 passe(s) décisive(s).</s>

<|assistant|>
        
        """
tokens = tokenizer(txt, return_tensors="pt")['input_ids'].to("cpu")
op = model.generate(tokens, max_new_tokens=200)
print(tokenizer.decode(op[0]))

NameError: name 'tokenizer' is not defined

In [61]:
# Sauvegarde du model finituné
model.save_pretrained("recomandation_joueur", safe_serialization=False)


In [62]:
!zip -r recomandation_joueur.zip '/workspaces/TestTecnique/finetuning/recomandation_joueur'

  adding: workspaces/TestTecnique/finetuning/recomandation_joueur/ (stored 0%)
  adding: workspaces/TestTecnique/finetuning/recomandation_joueur/adapter_config.json (deflated 55%)
  adding: workspaces/TestTecnique/finetuning/recomandation_joueur/adapter_model.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 9%)
  adding: workspaces/TestTecnique/finetuning/recomandation_joueur/README.md (deflated 66%)
